# BTC Prediction

In [6]:
import csv
import numpy as np
from datetime import date, datetime, timedelta
from lstm import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode
import cufflinks as cf
import yaml

CONFIG = "config/coinbase-BTCUSD.yml"
with open(CONFIG) as cfg:
    CONFIG = yaml.load(cfg, Loader=yaml.loader.SafeLoader)

MODEL_PATH = CONFIG["model"]
CSV_PATH = CONFIG["csv"]["out"]
TRAIN_TEST_RATE = CONFIG["splitRate"]
NUM_SEQ = CONFIG["seq"]
DUR_PRED = CONFIG["dur"]
TIME_UNIT = CONFIG.get("timeUnit") or 1

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]

machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(
    TRAIN_TEST_RATE, NUM_SEQ, DUR_PRED
)

dates = machine.dates_list
dates = [
    datetime.utcfromtimestamp(date / TIME_UNIT).date()
    for date in dates
]

base_date = min(dates)
end_date = max(dates)

user_date = None

while not user_date:
    try:
        user_date = date.fromisoformat(input(
            "What date you want to see? ("
            f"min: {base_date.isoformat()} "
            f"max: {end_date.isoformat()}) "
        ))
        if not (base_date <= user_date <= end_date):
            user_date = None
    except ValueError:
        continue

x_index = (user_date - base_date).days
act_data = machine.filtered_data
x = act_data.values[x_index - 15 : x_index, ]
x = np.array([x])

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(x)

pred = np.array([
    machine.scaler.inverse_transform(row)
    for row in pred
])[0]

for (c, key) in enumerate(['open', 'close', 'high', 'low']):
    pred_df = pd.DataFrame(
        {f'Pred': [
            item[c] for item in pred
        ]},
        index=pd.date_range(
            user_date,
            user_date + timedelta(days=len(pred) - 1)
        )
    )
    act_df = pd.DataFrame(
        {f'Act': [
            item[c] for item in act_data.values
        ]},
        index=dates
    )
    pd.concat([
        act_df,
#         pred_df
    ], axis=1).iplot(
        asFigure=False,
        kind='scatter',
        xTitle='Date',
        yTitle='Price',
        title=f'Price Prediction ({key})'
    )

What date you want to see? (min: 2016-09-01 max: 2019-04-24) 2019-04-24
